In [1]:
!pip install pandas scikit-learn matplotlib seaborn joblib


In [7]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load the dataset (Ensure your CSV file is in the same directory)
df = pd.read_csv("Dataset/modified_apartment_data.csv")

# Convert Time to Minutes Since Midnight
def time_to_minutes(time_str):
    h, m = map(int, time_str.split(":"))
    return h * 60 + m

df["Time_Minutes"] = df["Timings"].apply(time_to_minutes)
df.drop(columns=["Timings"], inplace=True)  # Drop original time column

# Create Positive Labels (Demand Exists)
df["Demand"] = 1  # All entries have demand

# Generate Negative Samples (No Demand)
np.random.seed(42)
negative_samples = pd.DataFrame({
    "Latitude": df["Latitude"] + np.random.uniform(-0.05, 0.05, size=len(df)),
    "Longitude": df["Longitude"] + np.random.uniform(-0.05, 0.05, size=len(df)),
    "Time_Minutes": np.random.randint(0, 1440, size=len(df)),  # Random times across the day
    "Demand": 0  # No demand
})

# Combine Positive & Negative Samples
df_final = pd.concat([df, negative_samples], ignore_index=True)

# Split Data for Training
X = df_final[["Latitude", "Longitude", "Time_Minutes"]]
y = df_final["Demand"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Model Evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save Model for Future Use
joblib.dump(model, "demand_prediction_model1.pkl")
print("Model saved as demand_prediction_model.pkl")


Accuracy: 0.9488888888888889
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.91      0.95       218
           1       0.92      0.98      0.95       232

    accuracy                           0.95       450
   macro avg       0.95      0.95      0.95       450
weighted avg       0.95      0.95      0.95       450

Model saved as demand_prediction_model.pkl


In [ ]:
# Load the saved model
model = joblib.load("demand_prediction_model1.pkl")

# Example Test Input
test_input = pd.DataFrame({
    "Latitude": [12.9716],   # Example: Bangalore
    "Longitude": [77.5946],
    "Time_Minutes": [8 * 60]  # 8:00 AM converted to minutes
})

# Predict Demand
prediction = model.predict(test_input)

# Output Result
print("Predicted Demand:", "High" if prediction[0] == 1 else "Low")



Predicted Demand: High


In [8]:
# Load the saved model
model = joblib.load("demand_prediction_model1.pkl")

# Example Test Input
test_input = pd.DataFrame({
    "Latitude": [12.9716],   # Example: Bangalore
    "Longitude": [77.5946],
    "Time_Minutes": [14 * 60]  # 8:00 AM converted to minutes
})

# Predict Demand
prediction = model.predict(test_input)

# Output Result
print("Predicted Demand:", "High" if prediction[0] == 1 else "Low")


Predicted Demand: Low
